In [42]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama  # Solution locale
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import gradio as gr
import torch
import warnings
warnings.filterwarnings('ignore')
import tqdm as notebook_tqdm
from langchain.prompts import PromptTemplate

In [23]:

pdf_files = ["cv_fr.pdf", "Valentin_Kocijancic_CV_2025 .pdf"]

# chargement pdf
documents = []
for file in pdf_files:
    loader = PyPDFLoader(file)
    docs = loader.load()
    documents.extend(docs)

# chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # taille des chunks
    chunk_overlap=200,    # chevauchement pour garder le contexte
    length_function=len
)
chunks = text_splitter.split_documents(documents)

print(f"Nombre de chunks créés : {len(chunks)}")

Nombre de chunks créés : 6


In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}  # 'cuda' si GPU
)

vector_store = FAISS.from_documents(chunks, embedding_model)

vector_store.save_local("faiss_index")

#print("Base FAISS créée et sauvegardée !")

In [25]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [43]:
llm = Ollama(
    model="phi3",
    temperature=0.1,  # Pour des réponses plus factuelles
    num_predict=256   # Limite la longueur des réponses
)

In [ ]:
template = """
Tu es un assistant intelligent spécialisé dans l'analyse de CV pour les recruteurs.
Tu dois répondre **en français** de manière claire, concise et professionnelle.

Tu disposes des informations suivantes issues d'une base de CV :
{context}

Consignes :
- Utilise uniquement les informations présentes dans le contexte pour répondre.
- Si la réponse n'est pas clairement indiquée, dis simplement : "L'information n'est pas disponible dans les CV."
- Si plusieurs candidats semblent correspondre, mentionne leurs **NOM ET PRÉNOM COMPLET** et explique brièvement pourquoi.
- Si un candidat se démarque particulièrement, indique-le clairement en donnant son **NOM ET PRÉNOM COMPLET** et justifie ton choix.

Question du recruteur : {question}

Réponse :
"""

prompt_fr = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_fr}
)

In [46]:
question = "Quel candidat cherche un stage en finance ?"
response = qa_chain({"query" : question})
print(response['result'])

Valentin Kocijančić est le candidat qui recherche un stage en finance, comme indiqué dans son CV où il mentionne explicitement qu'il "souhaite appliquer mes compétences techniques et mon esprit analytique au sein d’une équipe dynamique et stimulante." Il cherche également à commencer le stage dès le 13 avril 2026, ce qui correspond aux dates mentionnées dans la question du recruteur.
